In [2]:
import pandas as pd
from sklearn.model_selection import KFold
import pickle
from tcrpeg.TCRpeg import TCRpeg


In [3]:
# Create 5 Folds each from the allele and gene data from the PA-Transformers project. 
# IMPORTANT: Here we only used the test.tsv

# File paths
file_paths = [
    "data_for_inference/allele/beta/test.tsv",
    "data_for_inference/gene/beta/test.tsv"
]
precisions = ['allele', 'gene']

# Load, process, and split each file
for i, file_path in enumerate(file_paths):
    print(f"Processing {file_path}")
    
    # 1. Load the .tsv file
    df = pd.read_csv(file_path, sep='\t')
    
    # 2. Delete specific columns (replace 'column_to_delete' with actual column names)
    columns_to_delete = ['TCR_name', 'TRBV', 'TRBJ', 'TRBC', 'MHC', 'task']
    df.drop(columns=columns_to_delete, inplace=True, errors='ignore')
    
    # 3. Rename columns (replace with actual mappings)
    columns_to_rename = {
        'TRB_CDR3': 'CDR3.beta',
        'Binding': 'Label'
    }
    df.rename(columns=columns_to_rename, inplace=True)

    print('Satarting length= ', len(df))
    df = df[df['CDR3.beta'].apply(len) <= 30] 
    print('Length after removing CDR3.beta len > 30 = ', len(df))
    df = df[df['Epitope'].apply(len) <= 30] 
    print('Length after removing Epitope len > 30 = ', len(df))

    unique_chars = set(''.join(df['CDR3.beta']))  # Replace 'sequence_column' with your actual column name
    # print("Unique characters in sequences:", unique_chars)
    
    df_train = df['CDR3.beta'].values
    #create the TCRpeg class
    model = TCRpeg(hidden_size=64,num_layers = 3,load_data=True,embedding_path='pa_embeddings/TCRpeg/tcrpeg/data/embedding_32.txt',path_train=df_train)
    #create the TCRpeg model. 
    model.create_model()
    
    unmapped_chars = [ch for ch in unique_chars if ch not in model.aa2idx]
    print("Unmapped characters:", unmapped_chars)
    if unmapped_chars:
        
        # Create a regex pattern to match any of these characters
        pattern = f"[{''.join(unmapped_chars)}]"
        
        # Remove rows with any of the unmapped characters in 'CDR3.beta' column
        df = df[~df['CDR3.beta'].str.contains(pattern)]
        print('Length after removing unmapped chars= ', len(df))
    
    
    # 5. Generate 5 folds for cross-validation and save them in 'processed_data'
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    # Loop through each fold and save it as a separate file
    for fold, (_, fold_index) in enumerate(kf.split(df)):
        fold_data = df.iloc[fold_index]
        path_save_fold = f"processed_data/PA/{precisions[i]}_fold_{fold}.csv"
        fold_data.to_csv(path_save_fold, index=False)  # Saves each fold
        print(f"Saved fold {fold} as {precisions[i]}_fold_{fold}.csv in {path_save_fold}")



Processing data_for_inference/allele/beta/test.tsv
Satarting length=  59798
Length after removing CDR3.beta len > 30 =  59798
Length after removing Epitope len > 30 =  59455
Have loaded the data, total training seqs : 59455
Unmapped characters: []
Saved fold 0 as allele_fold_0.csv in processed_data/PA/allele_fold_0.csv
Saved fold 1 as allele_fold_1.csv in processed_data/PA/allele_fold_1.csv
Saved fold 2 as allele_fold_2.csv in processed_data/PA/allele_fold_2.csv
Saved fold 3 as allele_fold_3.csv in processed_data/PA/allele_fold_3.csv
Saved fold 4 as allele_fold_4.csv in processed_data/PA/allele_fold_4.csv
Processing data_for_inference/gene/beta/test.tsv
Satarting length=  53858
Length after removing CDR3.beta len > 30 =  53858
Length after removing Epitope len > 30 =  53484
Have loaded the data, total training seqs : 53484
Unmapped characters: []
Saved fold 0 as gene_fold_0.csv in processed_data/PA/gene_fold_0.csv
Saved fold 1 as gene_fold_1.csv in processed_data/PA/gene_fold_1.csv
Sav

In [ ]:
# run the inference_pa.py

In [4]:
# Inference with the respective folds of 'test.tsv'  ( file_path = f"processed_data/PA/{precision}_fold_{i}.csv" )
# and the embedding_path = f"models/PA/{precision}_embeddings.pkl"
! python inference_pa.py --split StrictTCR --dataset pMTnet --device gpu --gpu_id 0

You chose the dataset: pMTnet
The split method is: StrictTCR
/home/ubuntu/PA-Cancer-Immunotherapy/GNN/inference_pa.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  GTE.

In [ ]:
# Experiment: 
# Dataset: VDJdb  
# Model: pMTnet   model_path = f"models/{fixed_model}/{split}/{fixed_model}_{train_folds}_{i}.pth"
# 

In [1]:
! python inference_experiment.py --split StrictTCR --dataset VDJdb --device gpu --gpu_id 0

You chose the dataset: VDJdb
The split method is: StrictTCR
/home/ubuntu/PA-Cancer-Immunotherapy/GNN/inference_experiment.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

In [2]:
# =============================================================================
# Inference of Test data from BA (unseen) on the models gene and allele, those ones generated with train and validation data from BA aswell.
# A file inference_pa_one_model will be adapted for this purpose

! python inference_pa_one_model.py --device gpu --gpu_id 0

PROJECT_NAME: dataset-inference_GNN
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: frohoari (pa_cancerimmunotherapy). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /home/ubuntu/PA-Cancer-Immunotherapy/GNN/wandb/run-20241205_103656-ur4gvpur
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Experiment - GNN
wandb: ⭐️ View project at https://wandb.ai/pa_cancerimmunotherapy/dataset-inference_GNN
wandb: 🚀 View run at https://wandb.ai/pa_cancerimmunotherapy/dataset-inference_GNN/runs/ur4gvpur
torch.cuda.is_available:  True
Processing file:  ./data_for_inference/allele/beta/test.tsv
/home/ubuntu/PA-Cancer-Immunotherapy/GNN/inference_pa_one_model.py:211: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is 